In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# 1. Load Data
df = pd.read_csv('../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 2. Filter: from LOYAL customers
loyal_customers = df[df['Churn'] == 'No'].copy()

# 3. Select only "Service" columns
services = ['PhoneService', 'MultipleLines', 'InternetService',
            'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
            'TechSupport', 'StreamingTV', 'StreamingMovies']

basket = loyal_customers[services]

# 4. Convert text to Boolean (True/False)
def encode_units(x):
    if x in ['No', 'No internet service', 'No phone service']:
        return False
    else:
        return True

basket = basket.map(encode_units)

# 5. Run Apriori
frequent_itemsets = apriori(basket, min_support=0.05, use_colnames=True)

# 6. Generate Rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# 7. FILTERING
def is_interesting(consequent):
    # Convert frozenset to list
    items = list(consequent)
    if items == ['InternetService'] or items == ['PhoneService'] or items == ['MultipleLines']:
        return False
    return True

# Apply the filter
rules['is_interesting'] = rules['consequents'].apply(is_interesting)
interesting_rules = rules[rules['is_interesting'] == True].sort_values(['lift'], ascending=False)

# 8. Display the "Gold" Rules
print("--- TOP 5 UPSELLING OPPORTUNITIES (Based on Loyal Customers) ---")
cols_to_show = ['antecedents', 'consequents', 'support', 'confidence', 'lift']
display(interesting_rules[cols_to_show].head(5))

# Final output
print("\n--- AUTOMATED INSIGHT ---")
top_rule = interesting_rules.iloc[0]
ant = list(top_rule['antecedents'])[0]
cons = list(top_rule['consequents'])[0]
print(f"Strongest Pattern Found: Customers who have {ant} are very likely to buy {cons}.")
print(f"Strategy: Filter 'At Risk' customers who have {ant} but NOT {cons}, and offer them a discount on {cons}.")

--- TOP 5 UPSELLING OPPORTUNITIES (Based on Loyal Customers) ---


,antecedents,consequents,support,confidence,lift
13920,"(InternetService, StreamingTV, TechSupport, Mu...","(StreamingMovies, PhoneService, DeviceProtecti...",0.056243,0.484193,3.695007
13875,"(DeviceProtection, InternetService, PhoneServi...","(StreamingTV, TechSupport, MultipleLines)",0.056243,0.429204,3.695007
13941,"(StreamingMovies, PhoneService, DeviceProtecti...","(InternetService, StreamingTV, TechSupport, Mu...",0.056243,0.429204,3.695007
12757,"(StreamingTV, TechSupport, MultipleLines)","(StreamingMovies, PhoneService, DeviceProtecti...",0.056243,0.484193,3.695007
13986,"(StreamingTV, TechSupport, MultipleLines)","(DeviceProtection, InternetService, PhoneServi...",0.056243,0.484193,3.695007



--- AUTOMATED INSIGHT ---
Strongest Pattern Found: Customers who have InternetService are very likely to buy StreamingMovies.
Strategy: Filter 'At Risk' customers who have InternetService but NOT StreamingMovies, and offer them a discount on StreamingMovies.
